In [1]:
!git clone https://github.com/speckean/upar_dataset.git

fatal: destination path 'upar_dataset' already exists and is not an empty directory.


# Import Tools

In [2]:
import os
import pickle
import glob
import random

import numpy as np
import torch.utils.data as data
from PIL import Image

import torchvision.transforms as T
from torchvision import models

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [3]:
HOME = os.getcwd()
HOME

'C:\\Users\\USER\\Downloads\\LabML\\FashionDetection(Week3)\\UPAR(Soft-Biometric-Classification)'

# Dataset

## Get a data

In [4]:
!git clone https://github.com/speckean/upar_challenge.git
%cd upar_challenge
!pip install tqdm gdown requests

C:\Users\USER\Downloads\LabML\FashionDetection(Week3)\UPAR(Soft-Biometric-Classification)\upar_challenge


fatal: destination path 'upar_challenge' already exists and is not an empty directory.


In [5]:
!python3 download_datasets.py
%cd {HOME}
!mv upar_challenge/data .

C:\Users\USER\Downloads\LabML\FashionDetection(Week3)\UPAR(Soft-Biometric-Classification)


Traceback (most recent call last):
  File "download_datasets.py", line 9, in <module>
    import gdown
ModuleNotFoundError: No module named 'gdown'
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
current_dir_path = 'data/PA100k/release_data/release_data'
new_dir_path = 'data/PA100k/data'


if os.path.exists(current_dir_path):
    if not os.path.exists(new_dir_path):
        os.rename(current_dir_path, new_dir_path)
        print(f"Directory renamed from '{current_dir_path}' to '{new_dir_path}'")

    else:
        print(f"The target directory '{new_dir_path}' already exists.")
else:
    print(f"The directory '{current_dir_path}' does not exist.")

The directory 'data/PA100k/release_data/release_data' does not exist.


## Construct dataset/dataloader

In [7]:

class UPAR(data.Dataset):
  """
  Load UPAR dataset from pickle file

  split: whether to use train/val/trainval/test split
  partition: partition id 0-9
  root: path to datasets, original datasets must be in this directory
  data_path: path to UPAR pickle file
  transform: training data transforms
  target_transforms: evaluation data transforms
  """
  def __init__(self, split='train', partition=0, root=HOME, data_path='upar_dataset/UPAR/dataset_all.pkl', transform=None, target_transform=None):
    dataset_info = pickle.load(open(data_path, 'rb+'))
    self.dataset_info = dataset_info

    img_id = dataset_info.image_name
    attr_label = dataset_info.label

    assert split in dataset_info.partition.keys(), f'split {split} does not exist'

    self.dataset = 'UPAR'
    self.transform = transform  # data transforms during training
    self.target_transform = target_transform  # data transforms during testing

    self.root_path = root+"/data"  # path to datasets

    self.attr_id = dataset_info.attr_name  # attribute names
    self.attr_num = len(self.attr_id)  # number of attributes

		# load partition
    self.img_idx = dataset_info.partition[split]

    if isinstance(self.img_idx, list):
      self.img_idx = self.img_idx[partition]

    if isinstance(self.img_idx, list):
      self.img_idx = np.hstack(self.img_idx)

    self.img_idx = np.array([i for i in self.img_idx if not any(folder in img_id[i] for folder in ["RAP"])])


    self.img_num = self.img_idx.shape[0]
    self.img_id = [img_id[i] for i in self.img_idx]
    self.label = attr_label[self.img_idx]

    # set sub-dataset lengths to enable evaluation on sub-datasets
    self.sub_dataset_lengths = [len(d) for d in dataset_info.partition.test[partition]]


  def __getitem__(self, index):
      """
      get dataset item by index

      index: item index
      return: image data (img) with corresponding ground truth labels (gt_label), dataset id (did), and image path (imgname)
      """
      imgname, gt_label, imgidx = self.img_id[index], self.label[index], self.img_idx[index]
      did = self.dataset_info.dataset_ids[imgidx]
      imgpath = os.path.join(self.root_path, imgname)
      img = Image.open(imgpath)

      if self.transform is not None:
          img = self.transform(img)

      gt_label = gt_label.astype(np.float32)

      if self.target_transform is not None:
          gt_label = self.transform(gt_label)

      return img, gt_label, did, imgname


  def __len__(self):
    """
    get length of dataset
    """
    return len(self.img_id)


def get_transform(height, width):
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    train_transform = [
        T.Resize((height, width))
    ]

    train_transform += [
        T.Pad(10),
        T.RandomCrop((height, width)),
        T.RandomHorizontalFlip(),
    ]

    train_transform += [
        T.ToTensor(),
        normalize,
    ]
    train_transform = T.Compose(train_transform)

    valid_transform = T.Compose([
        T.Resize((height, width)),
        T.ToTensor(),
        normalize
    ])

    return train_transform, valid_transform

In [8]:
height = 224  # Example height
width = 224   # Example width
train_transform, valid_transform = get_transform(height, width)


# For training
train_dataset = UPAR(split='train', partition=0, transform=train_transform)

# For validation
val_dataset = UPAR(split='val', partition=0, transform=valid_transform)

# For testing
test_dataset = UPAR(split='test', partition=0, transform=valid_transform)


In [9]:
batch_size = 32  # Example batch size

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model Baseline

In [10]:
class UPARVGGModel(nn.Module):
    def __init__(self, num_attributes):
        super(UPARVGGModel, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        self.features = self.vgg16.features

        for param in self.features.parameters():
            param.requires_grad = False
        
        # VGG-16 features dimension before classifier is 512*7*7 assuming input size is 224x224
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_attributes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [11]:
num_attributes = 40
model = UPARVGGModel(num_attributes=num_attributes)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for batch, (X, y, _, _) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % 100 == 0:
            print(f"Batch {batch}, Loss: {loss.item()}")

    avg_loss = total_loss / len(dataloader)
    print(f"Average Training Loss: {avg_loss}")
    return avg_loss

def validate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y, _, _ in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Average Validation Loss: {avg_loss}")
    return avg_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    print("-" * 10)
    train_loss = train_one_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss = validate(model, val_loader, loss_fn, device)


Epoch 1/10
----------
Batch 0, Loss: 0.8904762268066406
Batch 100, Loss: 0.6772779226303101
Batch 200, Loss: 0.6788403987884521
Batch 300, Loss: 0.676496684551239
Batch 400, Loss: 0.6796216368675232
Batch 500, Loss: 0.6819654107093811
Batch 600, Loss: 0.6772779226303101
Batch 700, Loss: 0.6780591607093811
Batch 800, Loss: 0.6780591607093811
Batch 900, Loss: 0.6819654107093811
Batch 1000, Loss: 0.6796216368675232
Batch 1100, Loss: 0.676496684551239
Batch 1200, Loss: 0.6772779226303101
Batch 1300, Loss: 0.6796216368675232
Batch 1400, Loss: 0.6772779226303101
Batch 1500, Loss: 0.6772779226303101
Batch 1600, Loss: 0.6796216368675232
Batch 1700, Loss: 0.6811841726303101
Batch 1800, Loss: 0.680402934551239
Batch 1900, Loss: 0.6741529107093811
Batch 2000, Loss: 0.6827466487884521
Batch 2100, Loss: 0.6780591607093811
Batch 2200, Loss: 0.676496684551239
Batch 2300, Loss: 0.6757153868675232
Batch 2400, Loss: 0.6757153868675232
Batch 2500, Loss: 0.6772779226303101
Batch 2600, Loss: 0.681184172630

In [13]:
model_save_path = "vgg_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to vgg_model.pth
